In [1]:
import pandas as pd
import numpy as np
import urllib.request 
import os
import glob
import os.path
import csv
import re

In [22]:
pip3 install fsspec

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 138 kB 35.0 MB/s eta 0:00:01
You should consider upgrading via the '/spack/apps/linux-centos7-x86_64/gcc-8.3.0/python-3.9.2-uvcroioc4witkp6qf7mbebof5ix4wlb6/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
file_paths=[]
# after we loop through the folder and get every csv file, we go inside the csv file and grab links for each csv file
path_2022_03 = "/project/ll_774_951/uk_ru/twitter/data/2022-03/*.csv"
for fname in glob.glob(path_2022_03):
    file_paths.append(fname)
print(len(file_paths),'csv files in the 2022-03 folder')
#file_path

In [ ]:
#ath = "/project/ll_774_951/uk_ru/twitter/data/2022-03/ukraine_russia-2022-03-16-00.csv"
def combine_df(path,counter):
    # combine all files three columns (media_urls, rt_media_urls, q_media_urls) into one list
    # and store it to a csv file
    special_char = ",]['"
    image_links=pd.DataFrame()
    for csv in path: # try 2 files
        df = pd.read_csv(csv,engine='python', error_bad_lines=False)
        df_urls = df[['media_urls','rt_media_urls','q_media_urls']]
        df_all_urls = df_urls.melt(value_name='urls')[['urls']] # into single column
        download_1 = df_all_urls.loc[df_all_urls.urls.str.contains('http', na=False)]
        download_1 = download_1.replace("\[\'",'',regex=True)
        download_1 = download_1.replace("\'\]",'',regex=True)
        download_1 = download_1.replace("\,",'',regex=True)
        # download_list_1 = download_1["urls"].tolist()
        # downloads_1 = [''.join(x for x in string if not x in special_char) for string in download_list_1]
        #downloads_set =set(downloads_1)
        #downloads_list =list(downloads_set)
        image_links = pd.DataFrame(download_1)
        # blankIndex=[''] * len(image_links)
        # image_links.index=blankIndex
    # flat_image_links = list(np.concatenate(image_links)) # it was a list of lists
    # flat_image_links_set = set(flat_image_links)
    # flat_image_links_list = list(flat_image_links_set)
    # ls_csv_one_column = flat_image_links_list.to_csv(str(counter)+'.csv')
    ## need to change the above line for storing the csv to the correct path on HPC
    image_links.to_csv('/project/ll_774_951/uk_ru/twitter/one_column_urls/' + str(counter)+'.csv')
    return counter # listofurls

In [ ]:
def download_images(counter):
    df = pd.read_csv('/project/ll_774_951/uk_ru/twitter/one_column_urls/'+str(counter)+'.csv')
    # df = pd.read_csv('/project/ll_774_951/uk_ru/twitter/one_column_urls/1.csv')
    # df_urls = df[['urls']]
    # download_1 = df_all_urls.loc[df_all_urls.urls.str.contains('http', na=False)]
    # download_list_1 = download_1["urls"].tolist()
    # downloads_1 = [''.join(x for x in string if not x in special_char) for string in download_list_1]
    # downloads_set =set(downloads_1)
    # downloads_list =list(downloads_set)
    for i in range(len(df)):
        try:
            urllib.request.urlretrieve(df.loc[i, "urls"],'/project/ll_774_951/uk_ru/twitter/twitter_images2/'+os.path.basename(df.loc[i, "urls"]))
        except:
            pass

In [ ]:
batch_size = 50
counter = 0 # define a counter so that we can keep track which csv file we have download already
for i in range(0,len(file_paths),batch_size):
    counter+=1
    Batch = file_paths[i:i+batch_size]
    single_column_csv=combine_df(Batch,counter)
    download_images(single_column_csv)
    print('success'+str(counter))

In [ ]:
df = pd.read_csv('/project/ll_774_951/uk_ru/twitter/data/2022-03/ukraine_russia-2022-03-16-00.csv', engine='python', error_bad_lines=False)

In [70]:
image_links=pd.DataFrame()
special_char = ",]['"

df_urls = df[['media_urls','rt_media_urls','q_media_urls']]
df_all_urls = df_urls.melt(value_name='urls')[['urls']] # into single column
download_1 = df_all_urls.loc[df_all_urls.urls.str.contains('http', na=False)]
download_1 = download_1.replace("\[\'",'',regex=True)
download_1 = download_1.replace("\'\]",'',regex=True)
download_1 = download_1.replace("\,",'',regex=True)
        # download_list_1 = download_1["urls"].tolist()
        # downloads_1 = [''.join(x for x in string if not x in special_char) for string in download_list_1]
        #downloads_set =set(downloads_1)
        #downloads_list =list(downloads_set)
#image_links.append(download_1, ignore_index=True)
image_links = pd.DataFrame(download_1)
# image_links['urls'] = image_links['urls'].map(lambda x: x.lstrip(',').rstrip(','))
# blankIndex=[''] * len(image_links)
# image_links.index=blankIndex
    # flat_image_links = list(np.concatenate(image_links)) # it was a list of lists
    # flat_image_links_set = set(flat_image_links)
    # flat_image_links_list = list(flat_image_links_set)
    # ls_csv_one_column = flat_image_links_list.to_csv(str(counter)+'.csv')
    ## need to change the above line for storing the csv to the correct path on HPC
# image_links.to_csv('/project/ll_774_951/uk_ru/twitter/one_column_urls/' + str(counter)+'.csv')
print(image_links)

                                                      urls
23          http://pbs.twimg.com/media/FN7dxHOWUAAVya1.png
115      http://pbs.twimg.com/tweet_video_thumb/FN7eKtZ...
123         http://pbs.twimg.com/media/FN7eR5KXwAEcCfB.jpg
309         http://pbs.twimg.com/media/FN5l477XoAszsiM.jpg
315         http://pbs.twimg.com/media/FN7dlSmWQAU7F0Z.jpg
...                                                    ...
1066226  http://pbs.twimg.com/amplify_video_thumb/15036...
1066236  http://pbs.twimg.com/ext_tw_video_thumb/150388...
1066244     http://pbs.twimg.com/media/FNuL8iVXMAQfMUs.jpg
1066306  http://pbs.twimg.com/ext_tw_video_thumb/150382...
1066320     http://pbs.twimg.com/media/FNuL8iVXMAQfMUs.jpg

[71290 rows x 1 columns]


In [ ]:
    df = pd.read_csv('/project/ll_774_951/uk_ru/twitter/one_column_urls/1.csv')
    # df = pd.read_csv('/project/ll_774_951/uk_ru/twitter/one_column_urls/1.csv')
    # df_urls = df[['urls']]
    # download_1 = df_all_urls.loc[df_all_urls.urls.str.contains('http', na=False)]
    # download_list_1 = download_1["urls"].tolist()
    # downloads_1 = [''.join(x for x in string if not x in special_char) for string in download_list_1]
    # downloads_set =set(downloads_1)
    # downloads_list =list(downloads_set)
    for i in range(len(df)):
        print(df.loc[i, "urls"])


In [ ]:
tweet_id = df['tweetid']
tweet_id = tweet_id.dropna()
# for i in tweet_id:
#     url = "https://twitter.com/anyuser/status/" + i
print(tweet_id)